In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re
import json
import requests as rq # http requests
import bs4 as bs4 # beautiful soup for parsing
from bs4 import BeautifulSoup
import tqdm
from os import listdir
import glob
import urllib.request
from selenium import webdriver
from bs4 import BeautifulSoup

In [18]:
queries = ['olist']
n_pages = 4

In [19]:
list_pages = np.arange(1,(n_pages*48), 48)

In [20]:
list_pages

array([  1,  49,  97, 145])

In [26]:
url = 'https://lista.mercadolivre.com.br/_Desde_{page}_Loja_olist'

In [28]:
## Scrapping Pages
for page in list_pages:
    urll = url.format(page=page)
    response = rq.get(urll)
    with open("./raw_data/page_{}.html".format(page),'w+') as output:
        output.write(response.text)
        time.sleep(2)

In [50]:
for page in list_pages:
    with open('./raw_data/page_{}.html'.format(page),'r+') as inp:
        page_html = inp.read()
        parsed = bs4.BeautifulSoup(page_html)
        tags = parsed.findAll('a', {"class": "item-link item__js-link"})
        
        for e in tags:
            link = e['href']
            title = e.find('img')['alt']

            with open('parsed_products_links.json','a+') as output:
                data = {'link':link,'title':title}
                output.write('{}\n'.format(json.dumps(data)))
                

In [5]:
df = pd.read_json('./parsed_products_links.json', lines=True)

In [6]:
links = df.link.unique()

In [70]:
# Scrapping products
for i,link in enumerate(links):
    urll = link
    response = rq.get(urll)
    with open("./raw_data_products/product_{}.html".format(i),'w+') as output:
        output.write(response.text)


In [ ]:
with open("parsed_product_info.json","w+") as output:
    for product in tqdm.tqdm_notebook(sorted(glob.glob("./raw_data_products/product*"))):
        with open(product,"r+") as inp:
            page_html = inp.read()
            parsed = bs4.BeautifulSoup(page_html,"html.parser") 
            
            questions = parsed.find_all('p', attrs={"class":'ui-pdp-color--BLACK ui-pdp-size--SMALL ui-pdp-family--REGULAR ui-pdp-qadb__questions-list__question__label'}) 
            print(questions)

In [3]:
from selenium.common.exceptions import WebDriverException

In [2]:
image_source = '/home/vinicius/Área de Trabalho/codes/megahack/raw_data_products/'

In [ ]:
## LOOP

driver = webdriver.Chrome('/home/vinicius/Downloads/chromedriver_linux64/chromedriver')
data = {
    'title': [],
    'questions': [],
    'answers': [],
    'time_answer': []
}

for file in sorted(listdir(image_source)):
    filename = 'file://{}{}'.format(image_source,file)
    driver.get(filename)
    time.sleep(2)

    try:
        driver.find_element_by_xpath('//*[@id="root-app"]/div/div[2]/div[3]/div[1]/div[2]/div/div/div[3]/span').click()
        soup = BeautifulSoup(driver.page_source,"lxml")
        #print("Catch on method 2")
    except Exception:
        #print('Not catch on method 1, raising Exception Method')
        try:
            driver.find_element_by_xpath('/html/body/main/div[2]/div[1]/div[1]/section[5]/a').click()
            soup = BeautifulSoup(driver.page_source,"lxml")
            #print('Catch on method 2')
        except:
            #print('Not found questions on file {}'.format(filename))
            soup = BeautifulSoup(driver.page_source,"lxml")

    ## Here the page is full loaded
    questions = soup.find_all('p', {'class': 'ui-pdp-color--BLACK ui-pdp-size--SMALL ui-pdp-family--REGULAR ui-pdp-qadb__questions-list__question__label'})
    questions2 = soup.find_all('article', {'class': 'questions__item questions__item--question'})

    answers = soup.find_all('p', {'class': 'ui-pdp-color--BLACK ui-pdp-size--SMALL ui-pdp-family--REGULAR ui-pdp-qadb__questions-list__answer-item__answer'})
    answers2 = soup.find_all('article', {'class': 'questions__item questions__item--answer'})

    time2 = soup.find_all('time', {'class':'questions__time'})
    
    product_name = soup.find_all('h1', {'class': 'ui-pdp-title'})
    product_name2 = soup.find_all('header', {'class': 'item-title'})

    for e in product_name2:
        product_name2 = x.find('h1').text.strip()

    if len(questions)>0 and len(answers)>0:
        #print('cai no questions 1')
        for question, answer in zip(questions, answers):
            data['title'].append(product_name[0].text)
            data['questions'].append(question.text)
            data['answers'].append(answer.text)
            data['time_answer'].append(None)

    elif len(questions2) >0 and len(answers2) > 0:
        #print('cai na questions 2')
        for question, answer in zip(questions2,answers2):
            data['title'].append(product_name2)
            data['questions'].append(question.find('p').text)
            data['answers'].append(answer.find('p').text)
            data['time_answer'].append(answer.find('time').text.strip())
    



In [214]:
df_out = pd.DataFrame(data)

In [215]:
df_out.to_json('parsed_products_info.json',orient='records',lines=True)

In [23]:
image_source = '/home/vinicius/Área de Trabalho/codes/hackathon-api/scrapping/raw_data_products/'
file = 'product_2.html'
filename = 'file://{}{}'.format(image_source,file)

In [24]:
driver = webdriver.Chrome('/home/vinicius/Downloads/chromedriver_linux64/chromedriver')
driver.get(filename)
time.sleep(2)
question_id = 0
product_id = 0
i = 0

try:
    driver.find_element_by_xpath('//*[@id="root-app"]/div/div[2]/div[3]/div[1]/div[2]/div/div/div[3]/span').click()
    soup = BeautifulSoup(driver.page_source,"lxml")
    print("Catch on method 2")
except Exception:
    print('Not catch on method 1, raising Exception Method')
    try:
        driver.find_element_by_xpath('/html/body/main/div[2]/div[1]/div[1]/section[5]/a').click()
        soup = BeautifulSoup(driver.page_source,"lxml")
        print('Catch on method 2')
    except:
        print('Not found questions button on file {}'.format(filename))
        soup = BeautifulSoup(driver.page_source,"lxml")
        
## Here the page is full loaded
questions = soup.find_all('p', {'class': 'ui-pdp-color--BLACK ui-pdp-size--SMALL ui-pdp-family--REGULAR ui-pdp-qadb__questions-list__question__label'})
questions2 = soup.find_all('article', {'class': 'questions__item questions__item--question'})

answers = soup.find_all('p', {'class': 'ui-pdp-color--BLACK ui-pdp-size--SMALL ui-pdp-family--REGULAR ui-pdp-qadb__questions-list__answer-item__answer'})
answers2 = soup.find_all('article', {'class': 'questions__item questions__item--answer'})

product_name = soup.find_all('h1', {'class': 'ui-pdp-title'})
product_name2 = soup.find_all('header', {'class': 'item-title'})

description = soup.find_all('li', {'class': 'ui-pdp-features__item'})
description2 = soup.find_all('div', {'class': 'item-description__text'})

price = soup.find_all('div', {'class': 'ui-pdp-price mt-16 ui-pdp-price--size-large'})
price2 = soup.find_all('fieldset', {'class': 'item-price'})

for e in price:
    price = e.find('span', {'class': 'price-tag-fraction'}).text
    
for e in price2:
    cents = e.find('span', {'class':'price-tag-cents'})
    
    if cents:
        price2 = '{}.{}'.format(e.find('span', {'class':'price-tag-fraction'}).text, e.find('span', {'class':'price-tag-cents'}).text)
    else:
        price2 = '{}'.format(e.find('span', {'class':'price-tag-fraction'}).text)
        

for e in product_name2:
    product_name2 = e.find('h1').text.strip()

description_list = []
for e in description:
    description_list.append(e.text)
description = ' '.join(description_list)


for e in description2:
    description2 = e.text.strip()

data_questions = {
    'id': [],
    'product_id': [],
    'title': [],
    'questions': [],
    'status': [],
    'answers': [],
    'is_good': [],
}

data_products = {
    "id": [], # primary key
    "name": [], # string
    "quantity": [], #int
    "size": [], #tuple
    "color": [], #list
    "price": [], #float
    "weight": [], #float
    "image": [], #list
    "description": [],
}

if len(product_name)>0:
    data_products['id'].append(i)
    data_products['name'].append(product_name[0].text)
    data_products['quantity'].append(5) #valor generico
    data_products['size'].append(None)
    data_products['color'].append(None)
    data_products['price'].append(price)
    data_products['weight'].append(None)
    data_products['image'].append(None)
    data_products['description'].append(description)

elif len(product_name2)>0:
    data_products['id'].append(i)
    data_products['name'].append(product_name2)
    data_products['quantity'].append(5) #valor generico
    data_products['size'].append(None)
    data_products['color'].append(None)
    data_products['price'].append(price2)
    data_products['weight'].append(None)
    data_products['image'].append(None)
    data_products['description'].append(description2)
    


## Save Questions Info
if len(questions)>0 and len(answers)>0:
    print('cai no questions 1')
    for question, answer in zip(questions, answers):
        data_questions['id'].append(question_id)
        data_questions['product_id'].append(product_id)
        data_questions['questions'].append(question.text)
        data_questions['answers'].append(answer.text)
        data_questions['is_good'].append(np.nan)
        data_questions['status'].append('manual')
        
        question_id+=1
        
elif len(questions2) >0 and len(answers2) > 0:
    print('cai na questions 2')
    for question, answer in zip(questions2,answers2):
        data_questions['id'].append(question_id)
        data_questions['product_id'].append(product_id)
        data_questions['questions'].append(question.find('p').text)
        data_questions['answers'].append(answer.find('p').text)
        data_questions['is_good'].append(np.nan)
        data_questions['status'].append('manual')
        
        question_id+=1

driver.close()

Not catch on method 1, raising Exception Method
Not found questions button on file file:///home/vinicius/Área de Trabalho/codes/hackathon-api/scrapping/raw_data_products/product_2.html


In [25]:
data_products

{'id': [0],
 'name': ['Máquina de costura Lenoxx Multi Points PSM101 branca 110V/220V'],
 'quantity': [5],
 'size': [None],
 'color': [None],
 'price': ['341'],
 'weight': [None],
 'image': [None],
 'description': ['Mecânica.  Com 12 pontos diferentes. Tipos de pontos que incluem:  caseado, ponto invisível, reto, ziguezague. Passador manual.']}

In [ ]:
driver = webdriver.Chrome('/home/vinicius/Downloads/chromedriver_linux64/chromedriver')
data_questions = {
    'id': [],
    'product_id': [],
    'questions': [],
    'status': [],
    'answers': [],
    'is_good': [],
}

data_products = {
    "id": [], # primary key
    "name": [], # string
    "quantity": [], #int
    "size": [], #tuple
    "color": [], #list
    "price": [], #float
    "weight": [], #float
    "image": [], #list
    "description": [],
}
question_id = 0


for i,file in enumerate(sorted(listdir(image_source))):
    filename = 'file://{}{}'.format(image_source,file)
    driver.get(filename)
    time.sleep(2)

    try:
        driver.find_element_by_xpath('//*[@id="root-app"]/div/div[2]/div[3]/div[1]/div[2]/div/div/div[3]/span').click()
        soup = BeautifulSoup(driver.page_source,"lxml")
        print("Catch on method 2")
    except Exception:
        print('Not catch on method 1, raising Exception Method')
        try:
            driver.find_element_by_xpath('/html/body/main/div[2]/div[1]/div[1]/section[5]/a').click()
            soup = BeautifulSoup(driver.page_source,"lxml")
            print('Catch on method 2')
        except:
            print('Not found questions button on file {}'.format(filename))
            soup = BeautifulSoup(driver.page_source,"lxml")

    ## Here the page is full loaded
    questions = soup.find_all('p', {'class': 'ui-pdp-color--BLACK ui-pdp-size--SMALL ui-pdp-family--REGULAR ui-pdp-qadb__questions-list__question__label'})
    questions2 = soup.find_all('article', {'class': 'questions__item questions__item--question'})

    answers = soup.find_all('p', {'class': 'ui-pdp-color--BLACK ui-pdp-size--SMALL ui-pdp-family--REGULAR ui-pdp-qadb__questions-list__answer-item__answer'})
    answers2 = soup.find_all('article', {'class': 'questions__item questions__item--answer'})

    product_name = soup.find_all('h1', {'class': 'ui-pdp-title'})
    product_name2 = soup.find_all('header', {'class': 'item-title'})

    description = soup.find_all('li', {'class': 'ui-pdp-features__item'})
    description2 = soup.find_all('div', {'class': 'item-description__text'})

    price = soup.find_all('div', {'class': 'ui-pdp-price mt-16 ui-pdp-price--size-large'})
    price2 = soup.find_all('fieldset', {'class': 'item-price'})

    for e in price:
        price = e.find('span', {'class': 'price-tag-fraction'}).text

    for e in price2:
        cents = e.find('span', {'class':'price-tag-cents'})

        if cents:
            price2 = '{}.{}'.format(e.find('span', {'class':'price-tag-fraction'}).text, e.find('span', {'class':'price-tag-cents'}).text)
        else:
            price2 = '{}'.format(e.find('span', {'class':'price-tag-fraction'}).text)

    for e in product_name2:
        product_name2 = e.find('h1').text.strip()

    description_list = []
    for e in description:
        description_list.append(e.text)
    description = ' '.join(description_list)


    for e in description2:
        description2 = e.text.strip()


    if len(product_name)>0:
        data_products['id'].append(i)
        data_products['name'].append(product_name[0].text)
        data_products['quantity'].append(None) #valor generico
        data_products['size'].append(None)
        data_products['color'].append(None)
        data_products['price'].append(price)
        data_products['weight'].append(None)
        data_products['image'].append(None)
        data_products['description'].append(description)

    elif len(product_name2)>0:
        data_products['id'].append(i)
        data_products['name'].append(product_name2)
        data_products['quantity'].append(None) 
        data_products['size'].append(None)
        data_products['color'].append(None)
        data_products['price'].append(price2)
        data_products['weight'].append(None)
        data_products['image'].append(None)
        data_products['description'].append(description2)



    ## Save Questions Info
    if len(questions)>0 and len(answers)>0:
        print('cai no questions 1')
        for question, answer in zip(questions, answers):
            data_questions['id'].append(question_id)
            data_questions['product_id'].append(i)
            data_questions['questions'].append(question.text)
            data_questions['answers'].append(answer.text)
            data_questions['is_good'].append(np.nan)
            data_questions['status'].append('manual')

            question_id+=1

    elif len(questions2) >0 and len(answers2) > 0:
        print('cai na questions 2')
        for question, answer in zip(questions2,answers2):
            data_questions['id'].append(question_id)
            data_questions['product_id'].append(i)
            data_questions['questions'].append(question.find('p').text)
            data_questions['answers'].append(answer.find('p').text)
            data_questions['is_good'].append(np.nan)
            data_questions['status'].append('manual')

            question_id+=1

In [31]:
df_questions = pd.DataFrame(data_questions)

In [32]:
df_products = pd.DataFrame(data_products)

In [35]:
df_products.to_json('./json_files/products_table.json', orient='records',lines=True)

In [209]:
df_questions.to_json('questions_table.json', orient='records', lines=True)

In [34]:
df_questions

,id,product_id,questions,status,answers,is_good
0,0,0,O som é alto ?,manual,"Bom dia,Sim",NaN
1,1,0,Ela tem entrada auxiliar ?,manual,"Bom dia,somente via bluetooth",NaN
2,2,0,Essa caixinha tem entrada auxiliar?,manual,"Bom dia,somente via Bluetooth",NaN
3,3,0,Boa tarde qual a autonomia da bateria,manual,"Bom dia,na media de 2 horas dependendo do volu...",NaN
4,4,0,"Boa noite, essa caixa quando estiver carregand...",manual,"Olá, bom dia! sim, mais o ideal quer esteja de...",NaN
...,...,...,...,...,...,...
2184,2184,185,Qual o conector? p2?,manual,Olá sim conecta p2,NaN
2185,2185,185,Qual pino de alimentação dele? P2?,manual,"Olá, ele é P2 sim.",NaN
2186,2186,185,Ele é 7.1 ?,manual,Olá caro(a) cliente. O fone tem áudio 7.1 virt...,NaN
2187,2187,185,Da pra usar no celular?pra usar o fone e micro...,manual,"Olá sim da, vai com adaptador ps que funciona ...",NaN
